In [1]:
import torch

In [9]:
def shift_values_to_next_available_zero(a):
    b = (a > 0).int()
    b = torch.cat([torch.tensor([0], device=a.device), b])
    c = torch.arange(b.shape[0], device=a.device) * b
    c[-1] = 0
    d = c[1:]
    e = torch.roll(fill_zero_places(d), 1)
    e[0] = 0
    f = torch.zeros_like(a)
    f.scatter_add_(0, e.long(), a)
    return f

In [11]:
def fill_zero_places(a):
    a = torch.cat([torch.tensor([-1], device=a.device), a])
    nonzero_indices = torch.nonzero(a).squeeze()
    diff = torch.diff(torch.cat([nonzero_indices, torch.tensor([a.shape[0]], device=a.device)]))
    b = a[nonzero_indices].repeat_interleave(diff)
    b[b < 0] = 0
    c = b[1:]
    return c

In [5]:
def fill_zero_values_ascending_rest_with_zero(a):
    b = (a > 0).int()
    cumsum = torch.cumsum(1 - b, dim=0)
    c = (cumsum * (1 - b) - torch.maximum(b * cumsum, torch.zeros_like(cumsum)).cummax(dim=0)[0]) * (1 - b)
    return c

In [7]:
def find_one_zero_pattern(a):
    b = torch.roll(a.int(), -1)
    b[-1] = 1
    c = a - b
    d = (c == 1).int()
    return d

In [3]:
def fill_zero_values_descending_rest_with_zero(a):
    b = fill_zero_values_ascending_rest_with_zero(a)
    c = (b > 0).int()
    d = find_one_zero_pattern(c)
    e = d * b
    f = shift_values_to_next_available_zero(e)
    g = fill_zero_places(f)
    h = ((g + 1) - b) * c
    return h

In [12]:
a = torch.tensor([ 3, 10,  8, 11,  5, 13,  7, 14, 18, 15, 19, 20, 13, 21, 16, 24, 19, 24, 23, 22, 28, 22, 27, 26, 33, 32, 30, 32, 31, 36])
b, _ = torch.cummin(a.flip(0), dim=0)
b = b.flip(0)
c = torch.cat([(1 - (b[1:] - b[:-1]) > 0).int(), torch.tensor([0], device=a.device)])
d = a * c
e = ((d >= b).int() * c).int()
f = (1 - c)
g = fill_zero_values_descending_rest_with_zero(f)
h = d - g
i = ((h > b).int() * c | (a >= b).int() * c).int()

a, b, c, d, e, f, g, h, i, a  * i, a  * (1-i)

(tensor([ 3, 10,  8, 11,  5, 13,  7, 14, 18, 15, 19, 20, 13, 21, 16, 24, 19, 24,
         23, 22, 28, 22, 27, 26, 33, 32, 30, 32, 31, 36]),
 tensor([ 3,  5,  5,  5,  5,  7,  7, 13, 13, 13, 13, 13, 13, 16, 16, 19, 19, 22,
         22, 22, 22, 22, 26, 26, 30, 30, 30, 31, 31, 36]),
 tensor([0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0,
         1, 1, 0, 1, 0, 0]),
 tensor([ 0, 10,  8, 11,  0, 13,  0, 14, 18, 15, 19, 20,  0, 21,  0, 24,  0, 24,
         23, 22, 28,  0, 27,  0, 33, 32,  0, 32,  0,  0]),
 tensor([0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0,
         1, 1, 0, 1, 0, 0], dtype=torch.int32),
 tensor([1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
         0, 0, 1, 0, 1, 1]),
 tensor([0, 3, 2, 1, 0, 1, 0, 5, 4, 3, 2, 1, 0, 1, 0, 1, 0, 4, 3, 2, 1, 0, 1, 0,
         2, 1, 0, 1, 0, 0]),
 tensor([ 0,  7,  6, 10,  0, 12,  0,  9, 14, 12, 17, 19,  0, 20,  0, 23,  0, 20,
         20, 20, 27,  0, 26,  0, 31, 31,

#### Expiriment if there is a pattern (using NN)

In [194]:
import torch
import random

tensor_size = random.randint(100, 200)
num_ones = random.randint(40, min(100, tensor_size))
tensor = torch.zeros(tensor_size)
one_positions = random.sample(range(tensor_size), num_ones)
tensor[one_positions] = 1
start_index = torch.randint(0, 100, (1,))[0]
indices = torch.arange(start_index, start_index + tensor.shape[0])
ones = torch.ones(tensor.shape[0])
a = (indices * tensor)

tensor.sum(), tensor, indices, ones, a

(tensor(81.),
 tensor([1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
         0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0.,
         0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1.,
         0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
         1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0.,
         1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1.,
         1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1.,
         0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
         1., 1., 1., 1., 1., 1.]),
 tensor([ 91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
         105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
         119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132,
         133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 14

In [202]:
tensor.sum().int()

tensor(81, dtype=torch.int32)

In [216]:
decline_range = 5
declines = torch.randint(1, decline_range, (tensor.sum().int(),))
top_value_offset = torch.randint(0, tensor.sum().int() * decline_range, (1,))[0]

declines, top_value_offset

(tensor([2, 4, 4, 2, 1, 1, 4, 3, 2, 4, 2, 3, 1, 2, 4, 4, 1, 1, 4, 3, 3, 4, 4, 3,
         3, 2, 3, 1, 4, 2, 1, 2, 3, 4, 1, 4, 3, 3, 3, 4, 3, 2, 1, 1, 3, 2, 3, 4,
         1, 1, 1, 2, 4, 2, 2, 3, 1, 3, 4, 3, 4, 4, 4, 3, 4, 2, 2, 1, 1, 2, 4, 4,
         1, 2, 2, 2, 1, 2, 3, 4, 3]),
 tensor(271))

In [212]:
declines = torch.tensor([2, 4, 3, 2, 1])
top_value_offset = torch.tensor(323)

tensor([264])

In [221]:
indices = torch.nonzero(tensor > 0).flatten()
indices

tensor([  0,   1,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  19,  20,
         22,  23,  25,  26,  27,  28,  30,  32,  33,  37,  38,  39,  40,  42,
         44,  45,  48,  50,  51,  52,  53,  55,  58,  59,  60,  61,  62,  64,
         68,  72,  75,  76,  78,  79,  81,  82,  85,  87,  88,  90,  91,  94,
         98, 100, 103, 107, 108, 110, 112, 115, 116, 119, 121, 123, 124, 125,
        128, 129, 132, 135, 136, 144, 145, 146, 147, 148, 149])